<a href="https://colab.research.google.com/github/jessilver/Organizacao_Computadores/blob/develop/Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Faça uma cópia no seu drive

Importante: as instalações precisam ser feita toda vez que for iniciada uma nova seção no Colab

# Jessé Eliseu Nunes da Silva

# Como usar:

As implementações foram separadas e partes, para que a interface reconheça a parte que desesja utilizar bastas "ativa-la", trocando o valor da variavel de status de false para true.

## Testes:

### Teste Parte 1:

```
# --- Teste da Parte 1: Extensões e Erros ---

# 1. Teste de Divisão de Inteiros (DIV)
ADDI x1, x0, 40
ADDI x2, x0, 5
DIV x3, x1, x2  # Deve resultar em 8

# 2. Teste de Overflow (deve reportar ERRO e continuar)
ADDI x5, x0, 2147483647 # Valor máximo de inteiro 32-bit
ADDI x6, x0, 100
ADD x7, x5, x6  # Esta operação vai causar overflow

# 3. Teste de Divisão por Zero (deve reportar ERRO e continuar)
DIV x8, x1, x0 # x0 é sempre zero

# 4. Teste de Ponto Flutuante (deve executar após os erros)
# Usando os bits de 4.5 (1083179008) e 2.0 (1073741824)
ADDI x10, x0, 1083179008 # Carrega bits de 4.5
ADDI x11, x0, 1073741824 # Carrega bits de 2.0
ADD.S x12, x10, x11 # Soma de floats -> 6.5
MUL.S x13, x10, x11 # Multiplicação de floats -> 9.0
```

### Teste Parte 2:

```
# --- Teste da Parte 2: Funcionalidades Base ---

# Setup: Coloca valores iniciais nos registradores usando ADDI.
# (Usamos ADDI por conveniência, mas o foco do teste são as instruções abaixo)
ADDI x1, x0, 15
ADDI x2, x0, 10
ADDI x3, x0, 3

# 1. Soma os dois primeiros valores
# Requisito: ADD
ADD x4, x1, x2

# 2. Multiplica o resultado pelo terceiro valor
# Requisito: MUL
MUL x5, x4, x3

# 3. Armazena o resultado final (em x5) no endereço de memória 20
# Requisito: STORE (SW)
SW x5, 20

# 4. Carrega o valor do endereço de memória 20 para outro registrador (x6) para verificação
# Requisito: LOAD (LW)
LW x6, 20

# 5. Usa SUB para uma verificação final
# Requisito: SUB
SUB x7, x6, x5
```

### Teste Parte 3:

```
# --- Teste da Parte 3: Pipeline Básico ---

# Sequência de 5 instruções independentes para
# observar o preenchimento e esvaziamento do pipeline.

ADDI x1, x0, 1
ADDI x2, x0, 2
ADDI x3, x0, 3
ADDI x4, x0, 4
ADDI x5, x0, 5
```

# Interface

In [56]:
# ===================================================================
# Bloco 1: Interface do Usuário
# Este bloco contém todos os widgets e a lógica de clique dos botões.
# Ele chama as funções de simulação que serão definidas nos blocos abaixo.
# ===================================================================

import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Widgets de Entrada e Saída ---
code_area = widgets.Textarea(
    value="",
    placeholder="Escreva seu código assembly aqui...",
    description="Code:",
    layout=widgets.Layout(width="100%", height="200px")
)
output_area = widgets.Output()

# --- Widget de Seleção da Simulação ---
part_selector = widgets.RadioButtons(
    options=[
        ('Ciclo Único (Partes 1 & 2)', 'p1_2'),
        ('Parte 3: Pipeline', 'p3'),
        ('Parte 4: (A ser implementada)', 'p4'),
        ('Parte 5: (A ser implementada)', 'p5')
    ],
    description='Simular:',
    disabled=False
)

# --- Botões de Ação ---
run_button = widgets.Button(description="Run Program", button_style="success")
reg_button = widgets.Button(description="Show Registers", button_style="info")
mem_button = widgets.Button(description="Show Memory", button_style="info")
start_addr = widgets.IntText(value=0, description="Start Addr")
mem_range = widgets.IntText(value=10, description="Range")

# --- Handlers (Cérebros dos Botões) ---
def run_program_handler(b):
    # Limpa a área de output do widget
    with output_area:
        clear_output(wait=True)
        print("Inicializando hardware virtual...")

    # Zera os registradores e a memória para a nova simulação
    global regs, mem
    regs = RegisterFile()
    mem = SimpleMemory(64)
    code = code_area.value

    # Limpa a saída principal da célula e inicia o log
    # A recriação da UI evita que a interface desapareça
    clear_output(wait=True)
    display(ui)
    print("--- Log de Execução ---")

    # >>> LÓGICA PRINCIPAL: Decide qual simulação rodar <<<
    selected_part = part_selector.value
    if selected_part == 'p1_2':
        run_program_single_cycle(code, regs, mem) # Função definida nos Blocos 3 e 4
    elif selected_part == 'p3':
        # run_program_pipeline(code, regs, mem) # Função a ser definida no Bloco 5
        print("Lógica da Parte 3 (Pipeline) ainda não implementada neste esqueleto.")
    else:
        print(f"Lógica para a opção selecionada ainda não implementada.")

    print("--- Fim da Execução ---")
    with output_area:
        print("Programa executado. Use os botões para inspecionar o estado.")

def reg_button_handler(b):
    show_registers(regs)

def mem_button_handler(b):
    show_memory(mem, start=start_addr.value, rng=mem_range.value)

# --- Conecta os botões aos handlers ---
run_button.on_click(run_program_handler)
reg_button.on_click(reg_button_handler)
mem_button.on_click(mem_button_handler)

# --- Montagem e Exibição da Interface ---
# Inicialização global para evitar erros no primeiro clique em "Show"
regs, mem = RegisterFile(), SimpleMemory(64)

# Layout
top_controls = widgets.HBox([run_button, reg_button, mem_button, start_addr, mem_range])
ui = widgets.VBox([
    part_selector,
    code_area,
    top_controls,
    output_area
])

# Exibe a interface
display(ui)

# Componetes Comuns

In [52]:
# ===================================================================
# Bloco 2: Componentes Comuns
# Contém as classes, funções de formatação e de exibição que são
# usadas por todas as partes do simulador.
# ===================================================================

import pandas as pd
import struct

# --- Classes do Simulador ---
class SimpleMemory:
    def __init__(self, size_in_words=64):
        self.size = size_in_words
        self.mem = [0] * size_in_words
    def load_word(self, address):
        if 0 <= address < self.size: return self.mem[address]
        raise IndexError(f"Endereço de memória inválido: {address}")
    def store_word(self, address, value):
        if 0 <= address < self.size: self.mem[address] = value & 0xFFFFFFFF
        else: raise IndexError(f"Endereço de memória inválido: {address}")

class RegisterFile:
    def __init__(self):
        self.regs = [0] * 32
    def read(self, reg_num):
        if 0 <= reg_num < 32: return self.regs[reg_num]
        raise IndexError(f"Número de registrador inválido: x{reg_num}")
    def write(self, reg_num, value):
        if reg_num == 0: return
        if 0 < reg_num < 32: self.regs[reg_num] = value & 0xFFFFFFFF
        else: raise IndexError(f"Número de registrador inválido: x{reg_num}")

# --- Funções Auxiliares ---
def float_to_int_bits(f): return struct.unpack('I', struct.pack('f', float(f)))[0]
def int_bits_to_float(i):
    try: return struct.unpack('f', struct.pack('I', i & 0xFFFFFFFF))[0]
    except: return float('nan')
def to_signed32(value):
    value &= 0xFFFFFFFF
    return value - (1 << 32) if value & (1 << 31) else value
def format_instruction(instr):
    if not instr: return ""
    op = instr['op']
    if op in ["ADD", "SUB", "MUL", "DIV", "ADD.S", "MUL.S"]: return f"{op:<8} x{instr['rd']}, x{instr['rs1']}, x{instr['rs2']}"
    if op == "ADDI": return f"{op:<8} x{instr['rd']}, x{instr['rs1']}, {instr['imm']}"
    if op == "LW": return f"{op:<8} x{instr['rd']}, {instr['addr']}"
    if op == "SW": return f"{op:<8} x{instr['rs1']}, {instr['addr']}"
    return str(instr)

# --- Funções de Exibição ---
def show_registers(regs):
    output_widgets = []
    for i in range(0, 32, 8):
        start_reg, end_reg = i, i + 8
        register_values = [regs.read(j) for j in range(start_reg, end_reg)]
        data = {"Reg": [f"x{j}" for j in range(start_reg, end_reg)],
                "Valor Int": [to_signed32(val) for val in register_values],
                "Valor FP": [f"{int_bits_to_float(val):.4f}" for val in register_values]}
        df = pd.DataFrame(data).set_index("Reg")
        out = widgets.Output()
        with out: display(df)
        output_widgets.append(out)
    with output_area:
        clear_output(wait=True)
        display(widgets.HBox(output_widgets))

def show_memory(mem, start=0, rng=10):
    try:
        data = {"Endereço": list(range(start, start + rng)), "Valor": [mem.load_word(i) for i in range(start, start + rng)]}
        df = pd.DataFrame(data)
        with output_area: clear_output(wait=True); display(df)
    except IndexError as e:
        with output_area: clear_output(wait=True); print(f"Erro ao acessar memória: {e}")

# Parte 1


In [53]:
# ===================================================================
# Bloco 3: Lógica da Parte 2 (Simulador de Ciclo Único - Base)
#
# Este bloco define a estrutura fundamental do simulador, contendo
# apenas as instruções requeridas pela Parte 2: ADD, SUB, MUL, LW, SW.
# A lógica da Parte 1 será adicionada no bloco seguinte.
# ===================================================================

def execute_instruction_single_cycle(instr, regs, mem):
    """Versão base: Executa apenas instruções da Parte 2."""
    op = instr["op"]
    info = ""

    # --- Bloco de Operações Aritméticas (Base) ---
    if op in ["ADD", "SUB", "MUL"]:
        rs1_val = to_signed32(regs.read(instr["rs1"]))
        rs2_val = to_signed32(regs.read(instr["rs2"]))
        op_symbol = {'ADD': '+', 'SUB': '-', 'MUL': '*'}[op]

        if op == "ADD": result = rs1_val + rs2_val
        elif op == "SUB": result = rs1_val - rs2_val
        elif op == "MUL": result = rs1_val * rs2_val

        regs.write(instr["rd"], result)
        info = f"x{instr['rd']} = {rs1_val} {op_symbol} {rs2_val} -> {result}"

    # --- Bloco de Acesso à Memória ---
    elif op == "LW": # LOAD
        addr, rd = instr["addr"], instr["rd"]
        value = mem.load_word(addr)
        regs.write(rd, value)
        info = f"x{rd} = mem[{addr}] (Valor: {value})"
    elif op == "SW": # STORE
        addr, rs1 = instr["addr"], instr["rs1"]
        value = regs.read(rs1)
        mem.store_word(addr, value)
        info = f"mem[{addr}] = x{rs1} (Valor: {value})"
    else:
        raise ValueError(f"Instrução '{op}' não pertence ao conjunto base da Parte 2.")

    return info

def parse_program_single_cycle(asm_code):
    """Versão base: Reconhece apenas instruções da Parte 2."""
    program = []
    for line in asm_code.strip().splitlines():
        line = line.strip().split('#')[0]
        if not line: continue
        parts = line.replace(",", "").split()
        op = parts[0].upper()
        if op in ["ADD", "SUB", "MUL"]:
            instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "rs2": int(parts[3][1:])}
        elif op == "LW":
            instr = {"op": op, "rd": int(parts[1][1:]), "addr": int(parts[2])}
        elif op == "SW":
            instr = {"op": op, "rs1": int(parts[1][1:]), "addr": int(parts[2])}
        else:
            raise ValueError(f"Instrução '{op}' não reconhecida pelo parser base.")
        program.append(instr)
    return program

def run_program_single_cycle(asm_code, regs, mem):
    """Função principal de execução para o datapath de ciclo único."""
    try:
        program = parse_program_single_cycle(asm_code)
    except ValueError as e:
        print(f"ERRO DE SINTAXE: {e}")
        return

    for cycle, instr in enumerate(program, 1):
        formatted_instr = format_instruction(instr)
        try:
            info = execute_instruction_single_cycle(instr, regs, mem)
            print(f"Ciclo {cycle}: {formatted_instr:<25} -> {info}")
        except (ValueError, IndexError, OverflowError) as e:
            print(f"Ciclo {cycle}: {formatted_instr:<25} -> ERRO: {e}")

# Parte 2

In [54]:
# ===================================================================
# Bloco 4: Lógica da Parte 1 (Extensões para o Simulador)
#
# Este bloco REDEFINE as funções do Bloco 3, adicionando as
# funcionalidades da Parte 1: DIV, ADDI, Ponto Flutuante e
# detecção de overflow.
# Ao executar esta célula, estas versões completas passam a valer.
# ===================================================================

def execute_instruction_single_cycle(instr, regs, mem):
    """Versão completa: Executa instruções das Partes 1 e 2."""
    op = instr["op"]
    info = ""
    INT32_MIN, INT32_MAX = -2**31, 2**31 - 1

    if op in ["ADD", "ADDI", "SUB", "MUL", "DIV"]:
        rs1_val = to_signed32(regs.read(instr["rs1"]))
        rs2_val = instr["imm"] if op == "ADDI" else to_signed32(regs.read(instr["rs2"]))
        op_symbol = {'ADD': '+', 'ADDI': '+', 'SUB': '-', 'MUL': '*', 'DIV': '/'}[op]

        if op in ["ADD", "ADDI"]: result = rs1_val + rs2_val
        elif op == "SUB": result = rs1_val - rs2_val
        elif op == "MUL": result = rs1_val * rs2_val
        elif op == "DIV":
            if rs2_val == 0: raise ValueError("Divisão por zero")
            if rs1_val == INT32_MIN and rs2_val == -1: raise OverflowError(f"Overflow na divisão")
            result = int(rs1_val / rs2_val)

        if not (INT32_MIN <= result <= INT32_MAX) and op != "DIV":
            raise OverflowError(f"Overflow na operação: {rs1_val} {op_symbol} {rs2_val}")

        regs.write(instr["rd"], result)
        info = f"x{instr['rd']} = {rs1_val} {op_symbol} {rs2_val} -> {result}"

    elif op in ["ADD.S", "MUL.S"]:
        float1, float2 = int_bits_to_float(regs.read(instr["rs1"])), int_bits_to_float(regs.read(instr["rs2"]))
        op_symbol = '+' if op == "ADD.S" else '*'
        result_float = float1 + float2 if op == "ADD.S" else float1 * float2
        regs.write(instr["rd"], float_to_int_bits(result_float))
        info = f"x{instr['rd']} = {float1:.2f} {op_symbol} {float2:.2f} -> {result_float:.2f}"

    elif op == "LW":
        addr, rd = instr["addr"], instr["rd"]; value = mem.load_word(addr); regs.write(rd, value)
        info = f"x{rd} = mem[{addr}] (Valor: {value})"

    elif op == "SW":
        addr, rs1 = instr["addr"], instr["rs1"]; value = regs.read(rs1); mem.store_word(addr, value)
        info = f"mem[{addr}] = x{rs1} (Valor: {value})"

    else:
        info = f"Instrução desconhecida {op}"

    return info

def parse_program_single_cycle(asm_code):
    """Versão completa: Reconhece instruções das Partes 1 e 2."""
    program = []
    for line in asm_code.strip().splitlines():
        line = line.strip().split('#')[0]
        if not line: continue
        parts = line.replace(",", "").split()
        op = parts[0].upper()
        if op in ["ADD", "SUB", "MUL", "DIV", "ADD.S", "MUL.S"]:
            instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "rs2": int(parts[3][1:])}
        elif op == "ADDI":
            instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "imm": int(parts[3])}
        elif op == "LW":
            instr = {"op": op, "rd": int(parts[1][1:]), "addr": int(parts[2])}
        elif op == "SW":
            instr = {"op": op, "rs1": int(parts[1][1:]), "addr": int(parts[2])}
        else:
            raise ValueError(f"Instrução desconhecida: {line}")
        program.append(instr)
    return program

print("Funções de simulação da Parte 1 e 2 (versão completa) carregadas.")

Funções de simulação da Parte 1 e 2 (versão completa) carregadas.


# Parte 3

In [55]:
# ===================================================================
# Bloco 5: Lógica da Parte 3 (Simulador de Pipeline)
# Este bloco conterá a lógica para o processador com pipeline de 4 estágios.
# ===================================================================

def run_program_pipeline(asm_code, regs, mem):
    print("Executando simulação de PIPELINE...")

    # --- 1. Parse do Programa ---
    try:
        # Reusa o parser completo, pois o pipeline executará todas as instruções
        program = parse_program_single_cycle(asm_code)
    except ValueError as e:
        print(f"ERRO DE SINTAXE: {e}")
        return

    # --- 2. Estruturas de Dados do Pipeline ---
    pc = 0 # Program Counter
    pipeline_stages = {'IF': None, 'ID': None, 'EX': None, 'MEM_WB': None}
    history = []
    cycle = 0

    # --- 3. Loop Principal do Pipeline ---
    while pc < len(program) or any(s is not None for s in pipeline_stages.values()):
        cycle += 1
        prev_stages = pipeline_stages.copy()

        # --- Execução dos Estágios (ordem reversa) ---
        # Estágio MEM_WB (a ser implementado)
        if prev_stages['MEM_WB']: pass
        # Estágio EX (a ser implementado)
        pipeline_stages['MEM_WB'] = prev_stages['EX']
        # Estágio ID (a ser implementado)
        pipeline_stages['EX'] = prev_stages['ID']
        # Estágio IF (a ser implementado)
        pipeline_stages['ID'] = prev_stages['IF']
        if pc < len(program):
            pipeline_stages['IF'] = program[pc]
            pc += 1
        else:
            pipeline_stages['IF'] = None

        # --- 4. Registro do Histórico para Visualização ---
        history.append({
            'Ciclo': cycle,
            'IF': format_instruction(pipeline_stages['IF']),
            'ID': format_instruction(pipeline_stages['ID']),
            'EX': format_instruction(pipeline_stages['EX']),
            'MEM_WB': format_instruction(pipeline_stages['MEM_WB'])
        })

        if cycle > len(program) + 10: # Safety break
            print("Simulação parada: limite de ciclos atingido.")
            break

    # --- 5. Exibição da Tabela do Pipeline ---
    print("\n--- Tabela de Execução do Pipeline ---")
    if history:
        df = pd.DataFrame(history).set_index('Ciclo')
        with output_area:
             display(df.fillna(""))

    print(f"\nTotal de Ciclos (Pipeline): {cycle}")
    print(f"Total de Ciclos (Ciclo Único seria): {len(program)}")

# Parte 4

# Parte 5